### Celestrak Active Satellite Notebook

In [ ]:
import json
import requests

from sedaro import SedaroApiClient

#### Important: Read Before Running

This notebook makes changes to a scenario branche indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = 'NamFO_hSMyzY8s5kFw5hk'      # ID of the scenario branch

#### Instantiate Client
Instantiate the `SedaroApiClient` with our `API_KEY` and `HOST`, and fetch the desired branch.

In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY)
scenario_branch = sedaro.scenario(SCENARIO_BRANCH_ID)

### Fetch Active Satellite TLEs

In [ ]:
url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
tles = requests.get(url).text.splitlines()

### Create new agents with new TLEs

In [ ]:
def create_orbit(tle, i):
    _orbit = {
        'id': f'${i}',
        'initialStateDefType': 'TLE',
        'initialStateDefParams': {'tle': tle},
        'type': 'Orbit'
    }
    return _orbit

def create_agent(name, i):
    temp = {'orbit': f'${i}', 'name': name, 'type': 'Agent', 'peripheral': True, 'differentiatingState': {'targetType': 'SpaceTarget'}}
    return temp

n_sats = 1200 # int(len(tles)/3)
scenario_branch.crud(
    blocks=[
        create_orbit(tles[3*idx+1] + '\n' + tles[3*idx+2], f'${idx}') for idx in range(n_sats)
    ] + [
        create_agent(tles[3*idx], f'${idx}') for idx in range(n_sats)
    ]
)
print(f'Created {n_sats} orbits from TLEs and {n_sats} spacecraft digital twin agents.')

### Cleanup

In [ ]:
scenario_branch.crud(delete=[*scenario_branch.Orbit.get_all_ids(), *scenario_branch.Agent.get_all_ids()])